In [4]:
using LinearAlgebra
links1 = Dict{Int64, Vector{Int64}}(
    0 => [2],
    1 => [2, 3],
    2 => [3],
    3 => [],
    4 => [4]
)
links2 = Dict{Int64, Vector{Int64}}(
    0 => [2, 4],
    1 => [3, 0, 2],
    2 => [3, 4, 2],
    3 => [4],
    4 => []
)
links3 = Dict{Int64, Vector{Int64}}(
    0 => [0, 1],
    1 => [1, 0],
    2 => [3, 4],
    3 => [4, 2],
    4 => [3]
)
function get_markov(links::Dict{Int64, Vector{Int64}})
    n = length(links)
    B = zeros(Float64, n, n)

    # Construct B matrix
    for (i, outLink) in links
        if isempty(outLink)
            B[:, i+1] .= 1 / n
        else
            for j in outLink
                B[j+1, i+1] += 1 / length(outLink)
            end
        end
    end
    
    return B
end
function pagerank(links::Dict{Int64, Vector{Int64}}, c::Float64, tol::Float64 = 1e-6)
    n = length(links)
    B = get_markov(links)
    p = fill(1 / n, n)
    teleport = fill(1 / n, n)
    while true
        newP = c * B * p + (1 - c)*teleport
            if norm(newP - p) < tol
                break
        end
        p = newP
    end
    return p
end
function hits(links::Dict{Int64, Vector{Int64}}, tol::Float64 = 1e-6)
    B = get_markov(links)
    n = size(B, 1) # Number of nodes
    auth = ones(Float64, n) # Initialize authority scores
    hub = ones(Float64, n)
    auth /= norm(auth)
    hub /= norm(hub)
    HITSscore = zeros(Float64, n)
    while true
        newAuth = B' * hub
        newAuth /= norm(newAuth)
        newHub = B * newAuth
        newHub/= norm(newHub)
        if norm(newAuth - auth) < tol && norm(newHub - hub) < tol
            break
    end
        hub = newHub
        auth = newAuth
        HITSscore = 0.5*hub + 0.5*auth
    end
    return HITSscore
end

c = 0.85

ranks1 = pagerank(links1, c)
println("Page ranks of 1: ", ranks1)
c= 0.95
ranks2 = pagerank(links2, c)
println("Page ranks of 2: ", ranks2)
c= 0.85
ranks3 = pagerank(links3, c)
println("Page ranks of 3: ", ranks3)

# println("Page ranks of 3: ", ranks3)

println('\n')
HITSscore1 = hits(links1)
println("HITS score: ", HITSscore1)
HITSscore2 = hits(links2)
println("HITS score: ", HITSscore2)
HITSscore3 = hits(links3)
println("HITS score: ", HITSscore3)

Page ranks of 1: [0.06992838493600494, 0.06992838493600494, 0.1590872127042179, 0.23487236821599194, 0.4661836492077804]
Page ranks of 2: [0.10966156980118769, 0.08328698958718272, 0.2367087964284105, 0.18461938947166145, 0.38572325471155766]
Page ranks of 3: [0.2, 0.2, 0.14035125134585347, 0.2596487486541465, 0.19999999999999998]


HITS score: [0.29824945560423394, 0.29824945560423394, 0.6291634179218024, 0.4771111888458115, 0.09661188866595545]
HITS score: [0.29556440919132454, 0.10616512533157149, 0.38616822350034974, 0.4473169713626737, 0.5750446701605341]
HITS score: [2.105914627114022e-6, 2.105914627114022e-6, 0.3076856596695374, 0.5564398216495476, 0.564814178172687]
